In [120]:
import skimage as si
from scipy import misc
import numpy as np
import math

In [121]:
rgb_image = misc.imread("kaktus256.bmp")
rgb_image

# crvena = rgb_image[:,:,0]
# zelena = rgb_image[:,:,1]
# plava = rgb_image[:,:,2]

siva = np.array((rgb_image[:,:,0]//3 + rgb_image[:,:,1]//3 + rgb_image[:,:,2]//3),dtype='int16')

In [122]:
rgb_image[:,:,0] = siva
rgb_image[:,:,1] = siva
rgb_image[:,:,2] = siva

In [123]:
misc.imsave("gray_image.bmp",rgb_image)

In [124]:
broj_vertikalnih_blokova = siva.shape[0] // 8
broj_horizontalnih_blokova = siva.shape[1] // 8

In [135]:
#pravljenje matrice T
T = np.zeros((8, 8))
T[0] = 1/(math.sqrt(8))
for i in range(1, 8):
    for j in range(0, 8):
        T[i][j] = math.sqrt(2/8)*math.cos(((2*j+1)*i*math.pi)/(2*8))

In [136]:
#primer sa pdf-a
original1 = np.array([[154, 123, 123, 123, 123, 123, 123, 136],
              [192, 180, 136, 154, 154, 154, 136, 110],
              [254, 198, 154, 154, 180, 154, 123, 123],
              [239, 180, 136, 180, 180, 166, 123, 123],
              [180, 154, 136, 167, 166, 149, 136, 136],
              [128, 136, 123, 136, 154, 180, 198, 154],
              [123, 105, 110, 149, 136, 136, 180, 166],
              [110, 136, 123, 123, 123, 136, 154, 136]])
M1 = original1-128
D1 = T.dot(M1.dot(T.transpose()))
C1 = np.round(np.divide(D1,Q50))
C1

array([[ 10.,   4.,   2.,   5.,   1.,   0.,  -0.,  -0.],
       [  3.,   9.,   1.,   2.,   1.,  -0.,   0.,  -0.],
       [ -7.,  -5.,   1.,  -2.,  -1.,   0.,  -0.,   0.],
       [ -3.,  -5.,  -0.,  -1.,  -0.,   0.,  -0.,   0.],
       [ -2.,   1.,  -0.,  -0.,   0.,  -0.,   0.,  -0.],
       [ -0.,  -0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,  -0.,   0.,   0.,  -0.,  -0.,   0.,   0.],
       [ -0.,   0.,   0.,  -0.,   0.,   0.,   0.,   0.]])

In [137]:
# pravljenje kvantizacione matrice
Q50 = np.array([[16, 11, 10, 16, 24, 40, 51, 61],
              [12, 12, 14, 19, 26, 58, 60, 55],
              [14, 13, 16, 24, 40, 57, 69, 56],
              [14, 17, 22, 29, 51, 87, 80, 62],
              [18, 22, 37, 56, 68, 109, 103, 77],
              [24, 35, 55, 64, 81, 104, 113, 92],
              [49, 64, 78, 87, 103, 121, 120, 101],
              [72, 92, 95, 98, 112, 100, 103, 99]],dtype='int16')
Q50

# na osnovu izbora nivoa kvaliteta pravi se nova kvantizaciona matrica
nivo_kvaliteta = 10

# nivo kvaliteta treba da bude izmedju 1 i 99
if nivo_kvaliteta < 1:
    nivo_kvaliteta = 1

if nivo_kvaliteta > 99:
    nivo_kvaliteta = 100
    
if nivo_kvaliteta > 50:
    Q = (100-nivo_kvaliteta)/50*Q50
else:
    if nivo_kvaliteta < 50:
        Q = 50/nivo_kvaliteta*Q50
    else:
        Q = Q50

Q = np.array(Q, dtype='int16')    

for i in range(0, 8):
    for j in range(0, 8):
        if Q[i, j] > 255:
            Q[i, j] = 255
        if Q[i, j] < 1:
            Q[i, j] = 1

In [138]:
# obrada 8x8 matrica - kompresija
kompresovana_siva = np.zeros(siva.shape)

for i in range(0, broj_vertikalnih_blokova):
    for j in range(0, broj_horizontalnih_blokova):
        # za svaki blok...
        original = siva[i*8 : (i*8+8), j*8 : (j*8+8)]
        M = original-128
        D = T.dot(M.dot(T.transpose()))
        C = np.round(np.divide(D,Q))
        kompresovana_siva[i*8 : (i*8+8), j*8 : (j*8+8)] = C

In [139]:
# obrada 8x8  matrica - dekompresija
dekompresovana_siva = np.zeros(kompresovana_siva.shape)

for i in range(0, broj_vertikalnih_blokova):
    for j in range(0, broj_horizontalnih_blokova):
        # za svaki blok...
        C = kompresovana_siva[i*8 : (i*8+8), j*8 : (j*8+8)]
        R = np.multiply(C,Q)
        N = np.round((T.transpose()).dot(R.dot(T))) + 128
        dekompresovana_siva[i*8 : (i*8+8), j*8 : (j*8+8)] = N


rgb_image[:,:,0] = dekompresovana_siva
rgb_image[:,:,1] = dekompresovana_siva
rgb_image[:,:,2] = dekompresovana_siva
misc.imsave("gray_image_compressed10.bmp",rgb_image)